In [1]:
import logging
from math import floor

from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Convolution2D
from tensorflow.python.keras.layers import Dropout, Flatten, Dense
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras_preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

try:
    from pathlib import Path
except ImportError:
    from pathlib2 import Path

import pandas as pd
import os
import numpy as np

logger = logging.getLogger(__name__)

/home/s/work/car/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/s/work/car/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/s/work/car/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/s/work/car/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [2]:

IMAGE_INPUT_WIDTH=160
IMAGE_INPUT_HEIGHT=120
# This discards number of pixels from the top after scaling
TOP_MARGIN_IN_PIXELS=8


col_img = 'image'
col_steering = 'msg.steering'
col_throttle = 'msg.throttle'
col_image_ts = 'image_timestamp'
col_steernig_ts = 'steering_timestamp'
col_steering_scaled = 'steering_scaled'
col_throttle_scaled = 'throttle_scaled'

base_dir = Path("./data/")
def read_csv(path):
    df = pd.read_csv(path)
    for col in [col_img, col_steering, col_throttle, col_image_ts, col_steernig_ts]:
        assert col in df.columns, f'missing {col} in {path}'
    return df

# Read files and concatenate them

In [3]:
def load_and_merge_csvs(paths_csv):
    print('Loading {} csvs:\n{}'.format(len(paths_csv), '\n'.join(str(p) for p in paths_csv)))
    csvs = [read_csv(str(path)) for path in paths_csv]
    df = pd.concat(csvs, axis=0, ignore_index=True)
    print(f'loaded {df.shape}')
    return df

db_names =  [
        "sergem_robocar_20191009_v2",
        "sergem_robocar_20191106",
        "sergem_robocar_20191126_entrance",
        "sergem_robocar_20191127",
    ]
dfs = {
    db_name: load_and_merge_csvs(sorted(base_dir.glob(f"{db_name}/*.csv")))
    for db_name in db_names
}
# df1 = load_and_merge_csvs(sorted(base_dir.glob("sergem_robocar_20191003_v2/*.csv")))
# df2 = load_and_merge_csvs(sorted(base_dir.glob("*201911*/*.csv")))

Loading 3 csvs:
data/sergem_robocar_20191009_v2/robocar_recording_20191008_2016-02-11-16-43-19_0_f.csv
data/sergem_robocar_20191009_v2/robocar_recording_20191008_2016-02-11-16-46-19_1_f.csv
data/sergem_robocar_20191009_v2/robocar_recording_20191008_2016-02-11-16-49-19_2_f.csv
loaded (3968, 5)
Loading 23 csvs:
data/sergem_robocar_20191106/robocar_recording__2016-02-11-17-16-01_f.csv
data/sergem_robocar_20191106/robocar_recording__2016-02-11-17-33-19_1_f.csv
data/sergem_robocar_20191106/robocar_recording__2016-02-11-17-36-19_2_full.csv
data/sergem_robocar_20191106/robocar_recording__2016-02-11-17-39-19_3_full.csv
data/sergem_robocar_20191106/robocar_recording__2016-02-11-17-42-19_4_full.csv
data/sergem_robocar_20191106/robocar_recording__2016-02-11-17-57-53_0_f.csv
data/sergem_robocar_20191106/robocar_recording__2016-02-11-18-00-53_1_full.csv
data/sergem_robocar_20191106/robocar_recording__2016-02-11-18-03-53_2_f_1.csv
data/sergem_robocar_20191106/robocar_recording__2016-02-11-18-03-53_2

In [4]:
dfs[db_names[0]].head()

,image_timestamp,steering_timestamp,image,msg.steering,msg.throttle
0,1455209015392811874,1455209015396088310,robocar_recording_20191008_2016-02-11-16-43-19...,1368,1892
1,1455209015491766975,1455209015496871783,robocar_recording_20191008_2016-02-11-16-43-19...,1392,1904
2,1455209015590567337,1455209015596178809,robocar_recording_20191008_2016-02-11-16-43-19...,1400,1884
3,1455209015690528837,1455209015696991396,robocar_recording_20191008_2016-02-11-16-43-19...,1364,1896
4,1455209015790423254,1455209015798117888,robocar_recording_20191008_2016-02-11-16-43-19...,1324,1904


In [5]:
dfs[db_names[2]].head()

,image_timestamp,steering_timestamp,image,msg.steering,msg.throttle
0,1455209310596984722,1455209310599282726,robocar_recording__entrance__2019-11-26_0_f_0/...,1376,1536
1,1455209310700190735,1455209310709952591,robocar_recording__entrance__2019-11-26_0_f_0/...,1376,1536
2,1455209310798066735,1455209310801475668,robocar_recording__entrance__2019-11-26_0_f_0/...,1376,1536
3,1455209310898234854,1455209310901940868,robocar_recording__entrance__2019-11-26_0_f_0/...,1372,1532
4,1455209310998149328,1455209311002239454,robocar_recording__entrance__2019-11-26_0_f_0/...,1380,1528


# defining scaling
radio pwm has a certain range and the data is recorded as is. Now it's time to preprocess it to make it generic

In [7]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"



def plot_ranges(column, val_min=None, val_zero=None, val_max=None, update_layout_kwargs={}):
    graphs =[
        go.Scatter(x=column.index, y=column, mode='lines'),
    ]
    if val_min is not None:
        graphs.append(go.Scatter(x=column.index, y=val_min, mode='lines', name='min'))
    if val_zero is not None:
        graphs.append(go.Scatter(x=column.index, y=val_zero, mode='lines', name='zero',
                  line=dict(width=4,dash='dash')),
        )
    if val_max is not None:
        graphs.append(go.Scatter(x=column.index, y=val_max, mode='lines', name='max'),)
    fig = go.Figure()
    fig.add_traces(graphs)
    fig.update_layout(**update_layout_kwargs)
    fig.show()

In [8]:
def zeros_series_like(series):
    return series * 0

In [9]:
def rescale_steering(df,
            st_zero = 1372,
                     name='',
                     window_size_steering=20,
                     show=True,
                     
):
    df = df.copy()
    st_min = df[col_steering].rolling(window_size_steering, center=True, min_periods=1).quantile(quantile=0.98)
    st_max = df[col_steering].quantile(q=[0.01, 0.99])
    
    print(st_min, st_max)
    
    zero_col = zeros_series_like(df[col_steering])
    
    if show:
        plot_ranges(
            df[col_steering], 
            zero_col + st_min, 
            zero_col + st_zero, 
            zero_col + st_max, 
            update_layout_kwargs = {'title': 'steering initial' + name}
        )

    df[col_steering_scaled] = np.clip(np.interp(df[col_steering], (st_min, st_zero, st_max), (-1, 0, +1)), -1, 1)
    if show:
        plot_ranges(
            df[col_steering_scaled], 
            zero_col + -1, 
            zero_col + 0, 
            zero_col + 1, 
            update_layout_kwargs = {'title': 'steering scaled ' + name}
        )
    return df[col_steering_scaled]

In [10]:
# where we dont drive
def find_zeros(
    df,
    approx_zero_throttle = 1500,
    approx_zero_steering = 1400,
    th_zero_throttle = 150,
    th_zero_steering = 80,
    show=False,
):

    df = df.copy()

    is_idle = ( 
        (df[col_steering].rolling(30, center=True, min_periods=1).std() < 5)
        & (df[col_throttle].rolling(30, center=True, min_periods=1).std() < 5)
        & ((df[col_steering].rolling(5, center=True, min_periods=1).mean() - approx_zero_steering).abs() < th_zero_steering)
        & ((df[col_throttle].rolling(5, center=True, min_periods=1).mean() - approx_zero_throttle).abs() < th_zero_throttle)
    )
    df['idle'] = is_idle
    df['steering_zero'] = df[col_steering].where(is_idle > 0).interpolate(method='nearest', axis=0).ffill().bfill()
    df['throttle_zero'] = df[col_throttle].where(is_idle > 0).interpolate(method='nearest', axis=0).ffill().bfill()
    if show:
        plot_ranges(
            df[col_steering],
            df[col_throttle],
            df['throttle_zero'],
#             df['throttle_zero'] * is_idle,
            df['steering_zero'],
        )
    return df


In [11]:
df = dfs['sergem_robocar_20191127'].copy()
df = dfs['sergem_robocar_20191009_v2'].copy()
df = dfs['sergem_robocar_20191106'].copy()
df = dfs['sergem_robocar_20191126_entrance'].copy()

find_zeros(df, show=True)


,image_timestamp,steering_timestamp,image,msg.steering,msg.throttle,idle,steering_zero,throttle_zero
0,1455209310596984722,1455209310599282726,robocar_recording__entrance__2019-11-26_0_f_0/...,1376,1536,False,1372.0,1536.0
1,1455209310700190735,1455209310709952591,robocar_recording__entrance__2019-11-26_0_f_0/...,1376,1536,False,1372.0,1536.0
2,1455209310798066735,1455209310801475668,robocar_recording__entrance__2019-11-26_0_f_0/...,1376,1536,False,1372.0,1536.0
3,1455209310898234854,1455209310901940868,robocar_recording__entrance__2019-11-26_0_f_0/...,1372,1532,False,1372.0,1536.0
4,1455209310998149328,1455209311002239454,robocar_recording__entrance__2019-11-26_0_f_0/...,1380,1528,False,1372.0,1536.0
...,...,...,...,...,...,...,...,...
464,1455209385993208077,1455209386001796191,robocar_recording__entrance__2019-11-26_0_f_3/...,1232,1664,False,1372.0,1540.0
465,1455209386093187760,1455209386101759311,robocar_recording__entrance__2019-11-26_0_f_3/...,1232,1680,False,1372.0,1540.0
466,1455209386195152636,1455209386201809618,robocar_recording__entrance__2019-11-26_0_f_3/...,1232,1692,False,1372.0,1540.0
467,1455209386294281647,1455209386302668305,robocar_recording__entrance__2019-11-26_0_f_3/...,1232,1700,False,1372.0,1540.0


In [12]:
for name in db_names:
    dfs[name] = find_zeros(dfs[name], show=False)


In [13]:
def min_max_throttle(
    df,
    name='',
    show=False
):
    df = df.copy()
    throttle_max = (
        df[col_throttle]
        .where(df['idle'] == 0)
        .where(df[col_throttle] > df['throttle_zero'] + 200)
        .rolling(100, center=True, min_periods=100).quantile(0.80)
        .interpolate(method='nearest', axis=0).ffill().bfill()
        .fillna(df['throttle_zero'] + 500)
    )
    throttle_min = (
        df[col_throttle]
        .where(df['idle'] == 0)
        .where(df[col_throttle] < df['throttle_zero'] - 50)
        .rolling(500, center=True, min_periods=10).min()
    )

    throttle_min = throttle_min.where(throttle_min < df['throttle_zero'] - 200, df['throttle_zero'] - 500)
    df['throttle_min'] = throttle_min
    df['throttle_max'] = throttle_max
    if show:
        plot_ranges(
                    df[col_throttle], 
                    df['throttle_min'],
                    df['throttle_zero'],
                    df['throttle_max'],
                    update_layout_kwargs = {'title': 'throttle' + name}
                )
    return df

In [14]:
# df = dfs['sergem_robocar_20191127'].copy()
# df = dfs['sergem_robocar_20191009_v2'].copy()
# df = dfs['sergem_robocar_20191106'].copy()
# # df = dfs['sergem_robocar_20191126_entrance'].copy()

# _ = min_max_throttle(
#     df, 
#     show=True)

In [15]:
def min_max_steering(
    df,
    name='',
    show=False
):
    df = df.copy()
    df['steering_min'] = (
        df[col_steering]
        .rolling(1000, center=True, min_periods=100).quantile(0.001)
        .where(df['idle'] == 0)
        .interpolate(method='nearest', axis=0).ffill().bfill()
        .fillna(df['steering_zero'] - 250)
    )
    df['steering_max'] = (
        df[col_steering]
        
        .rolling(1000, center=True, min_periods=100).quantile(0.999)
        .where(df['idle'] == 0)
        .interpolate(method='nearest', axis=0).ffill().bfill()
        .fillna(df['steering_zero'] + 250)
    )

    if show:
        plot_ranges(
                    df[col_steering], 
                    df['steering_min'],
                    df['steering_zero'],
                    df['steering_max'],
                    update_layout_kwargs = {'title': 'steering ' + name}
                )
    return df

In [16]:
# df = dfs['sergem_robocar_20191127'].copy()
# df = dfs['sergem_robocar_20191009_v2'].copy()
# df = dfs['sergem_robocar_20191106'].copy()
# df = dfs['sergem_robocar_20191126_entrance'].copy()

# _ = min_max_steering(
#     df, 
#     show=True)

In [17]:
for name in db_names:
    dfs[name] = min_max_steering(min_max_throttle(dfs[name]))
    

In [18]:
def scale_minus1_plus1(x, xmin, xzero, xmax, do_clip):
    positive = x > xzero
    scale_positive = (x - xzero) / (xmax - xzero)
    scale_negative = (x - xzero) / (xzero - xmin)
    result = scale_positive.where(positive, scale_negative)
    if do_clip:
        result = result.clip(-1.,1.)
    
    return result

def scale_minus1_plus1_df(df):
    df = df.copy()
    df[col_steering_scaled] = scale_minus1_plus1(
        x = df[col_steering],
        xzero = df['steering_zero'],
        xmin = df['steering_min'],
        xmax = df['steering_max'],
        do_clip=True
    )
    df[col_throttle_scaled] = scale_minus1_plus1(
        x = df[col_throttle],
        xzero = df['throttle_zero'],
        xmin = df['throttle_min'],
        xmax = df['throttle_max'],
        do_clip=True
    )
    return df

In [19]:
# df = dfs['sergem_robocar_20191127'].copy()
# # df = dfs['sergem_robocar_20191009_v2'].copy()
# # df = dfs['sergem_robocar_20191106'].copy()
# # df = dfs['sergem_robocar_20191126_entrance'].copy()


# plot_ranges(
#     scale_minus1_plus1(
#         x = df[col_steering],
#         xzero = df['steering_zero'],
#         xmin = df['steering_min'],
#         xmax = df['steering_max'],
#         do_clip=True
#     )
# )

In [20]:
# plot_ranges(
#     scale_minus1_plus1(
#         x = df[col_throttle],
#         xzero = df['throttle_zero'],
#         xmin = df['throttle_min'],
#         xmax = df['throttle_max'],
#         do_clip=True
#     )
# )

In [21]:
for name in db_names:
    dfs[name] = scale_minus1_plus1_df(dfs[name])
    

In [22]:
for name in db_names:
    df = dfs[name]
    plot_ranges(df[col_steering], df['steering_min'], df['steering_zero'], df['steering_max'], 
                update_layout_kwargs = {'title': 'steering orig ' + name})
    plot_ranges(df[col_steering_scaled],
                update_layout_kwargs = {'title': 'steering scaled ' + name})
    plot_ranges(df[col_throttle], df['throttle_min'], df['throttle_zero'], df['throttle_max'], 
                update_layout_kwargs = {'title': 'throttle orig ' + name})
    plot_ranges(df[col_throttle_scaled],
                update_layout_kwargs = {'title': 'throttle scaled ' + name})

#### Discard if 

1) if there is a reverse in the future 3 sec

2) no throttle and no steering


In [23]:
def find_moments_before_reverse(
    df,
    col_target = 'reverse_in_the_future',
    threshold_ns = 3 * 1e9,
    th_reverse = -0.03,
):
    df = df.copy()
    df[col_target] = 0

    i_nearest_reverse = len(df) - 1
    ts_nearest_reverse = 1.0e30 # big value in the future
    res = []
    
    for i_cur in range(len(df) - 1, -1, -1):
        if df.iloc[i_cur][col_throttle_scaled] < th_reverse: 
            i_nearest_reverse = i_cur
            ts_nearest_reverse = df.iloc[i_nearest_reverse][col_image_ts]

        if ts_nearest_reverse - df.iloc[i_cur][col_image_ts] < threshold_ns:
            res.append(1)
        else:
            res.append(0)
            
    df[col_target] = res[::-1]
    return df[col_target]


In [24]:
def before_reversing_and_goodness(
    df,
    time_bofore_reverse_ns
):
    df = df.copy()
    df['reverse_in_the_future'] = find_moments_before_reverse(df, threshold_ns=time_bofore_reverse_ns)
    df['goodness'] = ((df['reverse_in_the_future'] == 0) & (df['idle'] == 0) & (df[col_throttle_scaled] > 0.3)) * 1
    return df



In [25]:
db_names

['sergem_robocar_20191009_v2',
 'sergem_robocar_20191106',
 'sergem_robocar_20191126_entrance',
 'sergem_robocar_20191127']

In [26]:
for name in [
    'sergem_robocar_20191009_v2',
    'sergem_robocar_20191106',
    'sergem_robocar_20191126_entrance',
]:
    dfs[name] = before_reversing_and_goodness(dfs[name], time_bofore_reverse_ns= 3*1e9)

In [27]:
for name in [
    'sergem_robocar_20191127',
]:
    dfs[name] = before_reversing_and_goodness(dfs[name], time_bofore_reverse_ns= 1)
    

In [28]:
df = dfs[db_names[3]]
go.Figure(data=[
    go.Scatter(x=df.index, y=df[col_steering_scaled], mode='lines', name='steeing_scaled'),
    go.Scatter(x=df.index, y=df[col_throttle_scaled], mode='lines', name='throttle_scaled'),
    go.Scatter(x=df.index, y=df['reverse_in_the_future'], mode='lines', name='reverse_in_the_future',
              line=dict(width=1,dash='dash')),
    go.Scatter(x=df.index, y=df['idle'], mode='lines', name='idle',
              line=dict(width=1,dash='dash')),
    go.Scatter(x=df.index, y=df['goodness'], mode='lines', name='goodness',
              line=dict(width=2,dash='dash')),

]).show()
df.size

674464

In [29]:
df[df['reverse_in_the_future']!=0]

,image_timestamp,steering_timestamp,image,msg.steering,msg.throttle,idle,steering_zero,throttle_zero,throttle_min,throttle_max,steering_min,steering_max,steering_scaled,throttle_scaled,reverse_in_the_future,goodness
10,1455211710996070288,1455211710996656846,robocar_recording_20191127_w_bwd__2016-02-11-1...,1396,1512,False,1376.0,1556.0,964.0,1948.0,1136.0,1665.964,0.068974,-0.074324,1,0
11,1455211711095999293,1455211711105261465,robocar_recording_20191127_w_bwd__2016-02-11-1...,1392,1500,False,1376.0,1556.0,964.0,1948.0,1136.0,1665.960,0.055180,-0.094595,1,0
12,1455211711198979943,1455211711205630155,robocar_recording_20191127_w_bwd__2016-02-11-1...,1396,1476,False,1376.0,1556.0,964.0,1948.0,1136.0,1665.956,0.068976,-0.135135,1,0
13,1455211711297490262,1455211711299086239,robocar_recording_20191127_w_bwd__2016-02-11-1...,1392,1204,False,1376.0,1556.0,964.0,1948.0,1136.0,1665.952,0.055182,-0.594595,1,0
14,1455211711396175266,1455211711397896712,robocar_recording_20191127_w_bwd__2016-02-11-1...,1396,1148,False,1376.0,1556.0,964.0,1948.0,1136.0,1665.948,0.068978,-0.689189,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39426,1455216859367377453,1455216859375723173,robocar_recording_20191127_w_bwd__2017-11-27-1...,1288,972,False,1392.0,1508.0,964.0,1976.0,1136.0,1664.000,-0.406250,-0.985294,1,0
39427,1455216859467405468,1455216859475802490,robocar_recording_20191127_w_bwd__2017-11-27-1...,1236,968,False,1392.0,1508.0,964.0,1976.0,1136.0,1664.000,-0.609375,-0.992647,1,0
39428,1455216859567381922,1455216859575620559,robocar_recording_20191127_w_bwd__2017-11-27-1...,1168,972,False,1392.0,1508.0,964.0,1976.0,1136.0,1664.000,-0.875000,-0.985294,1,0
39429,1455216859670015699,1455216859674674415,robocar_recording_20191127_w_bwd__2017-11-27-1...,1148,972,False,1392.0,1508.0,964.0,1976.0,1136.0,1664.000,-0.953125,-0.985294,1,0


## Saving filtered data

In [31]:
for db_name in db_names:
    df = dfs[db_name].copy()
    df[col_img] = df[col_img].map(lambda path: str(Path(db_name) / path))
    df = df[df['goodness'] != 0]
    print(df[df['goodness'] == 0].size)
    df.to_csv(base_dir / db_name / f"{db_name}.filteredcsv", index=False)

0
0
0
0


In [33]:
# df['image'].iloc[0]

# Loading filtered data 

In [34]:
filtered_csv_paths = sorted(base_dir.glob("**/*.filteredcsv"))
filtered_csv_paths

[PosixPath('data/sergem_robocar_20191009_v2/sergem_robocar_20191009_v2.filteredcsv'),
 PosixPath('data/sergem_robocar_20191106/sergem_robocar_20191106.filteredcsv'),
 PosixPath('data/sergem_robocar_20191126_entrance/sergem_robocar_20191126_entrance.filteredcsv'),
 PosixPath('data/sergem_robocar_20191127/sergem_robocar_20191127.filteredcsv')]

In [35]:
df_filtered = load_and_merge_csvs(filtered_csv_paths)

Loading 4 csvs:
data/sergem_robocar_20191009_v2/sergem_robocar_20191009_v2.filteredcsv
data/sergem_robocar_20191106/sergem_robocar_20191106.filteredcsv
data/sergem_robocar_20191126_entrance/sergem_robocar_20191126_entrance.filteredcsv
data/sergem_robocar_20191127/sergem_robocar_20191127.filteredcsv
loaded (48744, 16)


In [36]:
df_filtered.size

779904

In [37]:
df_filtered.head()

,image_timestamp,steering_timestamp,image,msg.steering,msg.throttle,idle,steering_zero,throttle_zero,throttle_min,throttle_max,steering_min,steering_max,steering_scaled,throttle_scaled,reverse_in_the_future,goodness
0,1455209015392811874,1455209015396088310,sergem_robocar_20191009_v2/robocar_recording_2...,1368,1892,False,1372.0,1524.0,972.0,1924.0,1144.0,1604.0,-0.017544,0.92,0,1
1,1455209015491766975,1455209015496871783,sergem_robocar_20191009_v2/robocar_recording_2...,1392,1904,False,1372.0,1524.0,972.0,1924.0,1144.0,1604.0,0.086207,0.95,0,1
2,1455209015590567337,1455209015596178809,sergem_robocar_20191009_v2/robocar_recording_2...,1400,1884,False,1372.0,1524.0,972.0,1924.0,1144.0,1604.0,0.120690,0.90,0,1
3,1455209015690528837,1455209015696991396,sergem_robocar_20191009_v2/robocar_recording_2...,1364,1896,False,1372.0,1524.0,972.0,1924.0,1144.0,1604.0,-0.035088,0.93,0,1
4,1455209015790423254,1455209015798117888,sergem_robocar_20191009_v2/robocar_recording_2...,1324,1904,False,1372.0,1524.0,972.0,1924.0,1144.0,1604.0,-0.210526,0.95,0,1


## split to training and validataion by indexes
It's just a proxy for time

In [38]:
is_valid = df_filtered.index % 1000 > 800
df_filtered['validation'] = is_valid * 1


In [39]:
go.Figure(data=[
    go.Scatter(x=df_filtered.index, y=df_filtered[col_steering_scaled], mode='lines', name='steeing_scaled'),
    go.Scatter(x=df_filtered.index, y=df_filtered[col_throttle_scaled], mode='lines', name='throttle_scaled'),
    go.Scatter(x=df_filtered.index, y=df_filtered['validation'], mode='lines', name='validation'),
]).show()

In [40]:
# shuffle to avoid issue: https://github.com/keras-team/keras-preprocessing/issues/205
df_train = df_filtered[df_filtered['validation'] == 0].sample(frac=1) 
df_val = df_filtered[df_filtered['validation'] == 1]
df_train.shape, df_val.shape

((39192, 17), (9552, 17))

# Training

In [41]:
datagen=ImageDataGenerator(rescale=1./255)

def create_generator(df):
    return datagen.flow_from_dataframe(dataframe=df, directory=str(base_dir),
        x_col=col_img,
        y_col=[col_steering_scaled, col_throttle_scaled],
        class_mode="multi_output",
        target_size=(IMAGE_INPUT_HEIGHT, IMAGE_INPUT_WIDTH),
        batch_size=32,
        shuffle=True,
    )
train_generator=create_generator(df_train)
val_generator=create_generator(df_val)


Found 39192 validated image filenames.
Found 9552 validated image filenames.


In [42]:
def visualize(imgs, y=None, pred=None):
    def format_(arr, idx):
        if arr is None:
            return "-"
        return ' '.join(['{:.3f}'.format(a.ravel()[idx]) for a in arr])
        
    _, axes = plt.subplots(8, 4, squeeze=False, figsize=(15,20), sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    for i, ax in enumerate(axes.flat):
        ax.imshow(imgs[i])
        
        text = "y {} \npred {}".format(format_(y, i), format_(pred, i))
        ax.text(20, 20, text, color='cyan')

In [43]:
np.set_printoptions(precision=3)

In [63]:
dbg_imgs, dbg_y = val_generator.next()
# visualize(dbg_imgs, dbg_y)

In [45]:
from keras_preprocessing.image.utils import load_img

In [46]:
dbg_y

[array([ 0.273, -0.038,  0.552, -0.962, -0.462, -0.962,  0.964, -0.075,
        -0.077,  0.985, -0.792, -0.585,  0.   , -0.2  ,  0.704, -0.969,
        -0.941, -1.   ,  0.91 , -0.369, -0.396, -0.189, -0.077, -0.892,
        -0.908,  0.   ,  0.   ,  0.06 , -0.338,  0.   ,  0.056,  0.507]),
 array([1.   , 0.929, 1.   , 0.992, 0.991, 0.97 , 1.   , 1.   , 0.974,
        0.92 , 0.96 , 1.   , 0.972, 0.91 , 1.   , 0.991, 0.992, 1.   ,
        1.   , 0.992, 0.98 , 1.   , 0.899, 0.38 , 0.957, 0.992, 0.965,
        0.992, 1.   , 0.961, 0.96 , 0.992])]

In [49]:
def default_linear():
    img_in = Input(shape=(120, 160, 3), name='img_in')
    x = img_in

    # Convolution2D class name is an alias for Conv2D
    x = Convolution2D(filters=24, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=32, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)

    x = Flatten(name='flattened')(x)
    x = Dense(units=100, activation='linear')(x)
    x = Dropout(rate=.1)(x)
    x = Dense(units=50, activation='linear')(x)
    x = Dropout(rate=.1)(x)
    # categorical output of the angle
    angle_out = Dense(units=1, activation='linear', name='angle_out')(x)

    # continous output of throttle
    throttle_out = Dense(units=1, activation='linear', name='throttle_out')(x)

    model = Model(inputs=[img_in], outputs=[
        angle_out, 
        throttle_out
    ])

    model.compile(optimizer='adam',
                  loss={
                      'angle_out': 'mean_squared_error',
                      'throttle_out': 'mean_squared_error'
                  },
                  loss_weights={
                      'angle_out': 0.97, 
                      'throttle_out': .03
                  }
                 )

    return model

In [50]:
model = default_linear()

In [51]:
use_early_stop = True
epochs = 12
saved_model_path = Path("./models/linear_20191129_wo_outdoor_data.model")
verbose = True
min_delta=.0005
patience=5

In [52]:
len(train_generator), train_generator.n, train_generator.batch_size, 40*32

(1225, 39192, 32, 1280)

In [53]:
saved_model_path.parent.mkdir(exist_ok=True, parents=True)

In [54]:
save_best = ModelCheckpoint(str(saved_model_path),
                            monitor='val_loss',
                            verbose=verbose,
                            save_best_only=True,
                            mode='min')

# stop training if the validation error stops improving.
early_stop = EarlyStopping(monitor='val_loss',
                           min_delta=min_delta,
                           patience=patience,
                           verbose=verbose,
                           mode='auto')

tb = TensorBoard()

callbacks_list = [save_best, tb]

if use_early_stop:
    callbacks_list.append(early_stop)

hist = model.fit_generator(
    train_generator,
    epochs=epochs,
    verbose=1,
    validation_data=val_generator,
    callbacks=callbacks_list)

Epoch 1/12
1224/1225 [============================>.] - ETA: 0s - loss: 0.1539 - angle_out_loss: 0.1574 - throttle_out_loss: 0.0403
Epoch 00001: val_loss improved from inf to 0.08868, saving model to models/linear_20191129_wo_outdoor_data.model
1225/1225 [==============================] - 191s 156ms/step - loss: 0.1539 - angle_out_loss: 0.1574 - throttle_out_loss: 0.0403 - val_loss: 0.0887 - val_angle_out_loss: 0.0912 - val_throttle_out_loss: 0.0083
Epoch 2/12
1224/1225 [============================>.] - ETA: 0s - loss: 0.0886 - angle_out_loss: 0.0908 - throttle_out_loss: 0.0181
Epoch 00002: val_loss improved from 0.08868 to 0.08531, saving model to models/linear_20191129_wo_outdoor_data.model
1225/1225 [==============================] - 187s 152ms/step - loss: 0.0886 - angle_out_loss: 0.0908 - throttle_out_loss: 0.0181 - val_loss: 0.0853 - val_angle_out_loss: 0.0877 - val_throttle_out_loss: 0.0075
Epoch 3/12
1224/1225 [============================>.] - ETA: 0s - loss: 0.0764 - angle_o

In [60]:
vis_imgs, vis_y = val_generator.next()
vis_pred = model.predict(vis_imgs)
visualize(vis_imgs, vis_y, vis_pred)

In [61]:
# steering
plt.plot(vis_y[0], label="manual")
plt.plot(vis_pred[0], label="prediction")
plt.legend()

In [62]:
# Throttle
plt.plot(vis_y[1])
plt.plot(vis_pred[1])